In [22]:
import os
import time
import json
import logging
from dotenv import load_dotenv
from typing import List, Dict
from pathlib import Path
from azure.identity import ClientSecretCredential
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from openai import AzureOpenAI

# ------------------ Load environment variables ------------------
load_dotenv()

# ------------------ Logging setup ------------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# ------------------ Azure Credentials ------------------
credential = ClientSecretCredential(
    tenant_id=os.getenv("AZURE_TENANT_ID"),
    client_id=os.getenv("AZURE_CLIENT_ID"),
    client_secret=os.getenv("AZURE_CLIENT_SECRET"),
)

# ------------------ Globals ------------------
TOKEN_USAGE = {}
TOTAL_API_CALL = 0
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# ------------------ Azure Document Intelligence Client ------------------
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

if not AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT or not AZURE_DOCUMENT_INTELLIGENCE_KEY:
    raise EnvironmentError("Missing Azure Document Intelligence credentials")

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT,
    credential=AzureKeyCredential(AZURE_DOCUMENT_INTELLIGENCE_KEY)
)

# ------------------ OCR Function ------------------
def process_file_new_ocr(file_path: str) -> List[Dict]:
    file_path = Path(file_path)
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    logging.info(f"Processing file: {file_path.name}")

    with open(file_path, "rb") as f:
        document_bytes = f.read()

    poller = document_intelligence_client.begin_analyze_document(
        model_id="prebuilt-read",
        body=document_bytes,
    )
    result = poller.result()

    attachment_list = []
    for page in result.pages:
        text = " ".join([line.content for line in page.lines])
        attachment_list.append({
            "title": file_path.name,
            "pagenum": page.page_number,
            "content": text
        })

    return attachment_list

# ------------------ Azure OpenAI LLM Clients ------------------
AzureChatOpenAI.model_rebuild()

def llm():
    access_token = credential.get_token("https://cognitiveservices.azure.com/.default").token
    return AzureChatOpenAI(
        azure_deployment=AZURE_OPENAI_DEPLOYMENT,
        api_version=os.getenv("AZURE_API_VERSION"),
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        openai_api_key=access_token,
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    )

def openai_llm():
    access_token = credential.get_token("https://cognitiveservices.azure.com/.default").token
    return AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("AZURE_API_VERSION"),
        api_key=access_token,
    )

def embeddings():
    access_token = credential.get_token("https://cognitiveservices.azure.com/.default").token
    return AzureOpenAIEmbeddings(
        model=os.getenv("EMBEDDING_AZURE_OPENAI_DEPLOYMENT"),
        azure_endpoint=os.getenv("EMBEDDING_AZURE_OPENAI_ENDPOINT"),
        openai_api_version=os.getenv("AZURE_API_VERSION"),
        api_key=access_token,
    )

# ------------------ OpenAI Call Utilities ------------------
def add_token_usage_logs(llm_output, message=""):
    token_usage_string = ""
    for key, value in llm_output.to_dict().get("usage", {}).items():
        token_usage_string += f"{key}: {value} | "
    logger.info(f"{message} {token_usage_string}")
    return llm_output.to_dict().get("usage", {}).get("total_tokens", 0)

def openai_call(sys_prompt, prompt_struc, deployment_name=AZURE_OPENAI_DEPLOYMENT, additional_message=""):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt_struc},
    ]
    return _call_openai(messages, deployment_name, additional_message)

def _call_openai(messages, deployment_name, additional_message):
    global TOTAL_API_CALL
    max_retries = 5
    for current_retry in range(max_retries):
        try:
            time.sleep(5)
            logger.info(f"Calling OpenAI API with deployment: {deployment_name}, Retry: {current_retry + 1}")
            start_time = time.time()
            response = openai_llm().chat.completions.create(
                model=deployment_name,
                messages=messages,
                temperature=0,
            )
            TOTAL_API_CALL += 1
            duration = round(time.time() - start_time, 3)
            output = response.choices[0].message.content
            if output:
                msg = f"OPENAI CALL MESSAGE: Retry {current_retry + 1} | Func: {additional_message} | Duration: {duration}s"
                total_tokens = add_token_usage_logs(response, message=msg)
                TOKEN_USAGE[additional_message] = TOKEN_USAGE.get(additional_message, 0) + total_tokens
                logger.info(f"Token Usage Summary: {TOKEN_USAGE}")
                print("\n🔹 OpenAI Response:\n", output)
                return output
        except Exception as e:
            logger.error(f"OpenAI Call Error: {e}")
    logger.warning("Maximum retries reached. No response returned.")
    return None

# ------------------ MAIN EXECUTION BLOCK ------------------
if __name__ == "__main__":
    test_file = "R&D Supplier Brief - Chocolate Coating for Murray Street.pdf"

    try:
        ocr_pages = process_file_new_ocr(test_file)
        extracted_text = "\n".join([p["content"] for p in ocr_pages])

        system_prompt = """
You are a domain-aware assistant that extracts structured product development attributes from customer R&D or supplier brief documents.

---

### 🎯 Your Role:
You are interpreting a **customer brief** — a document that includes both direct requirements and contextual information. Your job is to:
1. Extract **explicit** attributes: those that are **directly requested by the customer** in the brief.
2. Extract **inferred** attributes: high-confidence values **logically implied** by the brief’s context, product type, or regulatory standards, with a supporting `"note"`.

---

### 📌 Mandatory Instructions:
- Maximize the number of attributes extracted from the brief.
- **Always extract** the following fields if present — even if not explicitly requested:

['Material Code', 'Legislation', 'Cluster', 'Cluster_Label', 'Legal_Denomination', 'Legislation_Description', 'Min_Dry_Cocoa_Solids', 'Dry_Milk_Solids', 'MilkFat', 'Dry_Fat_Free_Cocoa_Solids', 'Typical_Chocolate_Liquor', 'Typical_Cocoa_Content', 'Total_Legal_Fat_Content', 'MilkFat_Chocolate_Part', 'Dry_Milk_Solids_On_Total_Production', 'Dry_Milk_Solids_Chocolate_Part', 'Typical_Nonfat_Milk_Solids', 'Typ_Nonfat_Cocoa_Sol_Choc_Part', 'Sum_Dry_Cocoa_And_Milk_Solids', 'Cocoa_Butter_Content', 'Alkalizing_Agent_K2CO3_DFFCS', 'Component', 'Item', 'Level', 'Material_Group', 'Material_Type', 'Parent_Material', 'Parent_Material_Label', 'Source_Generated_Field', 'components_Specifications', 'Sugars_g', 'Salt_g', 'Trans_Fatty_Acid_TFa_g', 'Energy_Value_Kcal', 'Energy_Value_Kj', 'Protein_g', 'Protein_DV_perc', 'Total_Carbohydrates_g', 'Total_Carbohydrates_DV_perc', 'Saturated_Fatty_Acid_g', 'Calories_From_Fat', 'Cholesterol_mg', 'Cholesterol_DV_perc', 'Total_Fat_DV_perc', 'Fibre_g', 'Dietary_Fibre_DV_perc', 'Vitamin_A_mcg', 'Vitamin_C_mg', 'Sodium_mg', 'Sodium_DV_perc', 'Iron_mg', 'Calcium_mg', 'Available_Carbohydrates_g', 'Total_Fat_g', 'Allergen_Statements', 'Contains_Milk_Proteins', 'Contains_Egg_Products', 'Contains_Soy_Proteins', 'Contains_Wheat', 'Contains_Rye', 'Contains_Fish', 'Contains_Crustacean_And_Shell_Fish', 'Contains_Hazelnuts_Almonds', 'Contains_Peanuts', 'Contains_Sulphite_E220_E227', 'Contains_Celery', 'Contains_Sesame_Products', 'Suitable_For_Vegetarians', 'Suitable_For_Vegans', 'Hazelnut_Oil_Almond_Oil', 'Contains_Sesame_Oil', 'Contains_Peanut_Oil', 'Contains_Mustard', 'Contains_Molluscs', 'Contains_Lupin', 'Contains_Buckwheat', 'Plant_BoM_Owner_Short', 'Underlying_Liquid', 'Dimensions_Vibration_Drops_EU_Short', 'Project_Number_Short', 'Dimensions_Production_Tools_Us', 'Material_Description', 'Base_Type', 'Moulding_Type', 'Product_Type', 'Colour_TF', 'Project_Manager', 'Dimensions_Vibration_Drops_EU', 'Project_Phase', 'Certification', 'Base_Colour', 'Additional_Colour', 'Kosher_Certificate', 'Country_Claim', 'Plant_BoM_Owner', 'Type_3_Short', 'Dosage_Per_200ml_Cold_Milk', 'Dosage_Per_200ml_Cold_Water', 'Dosage_Per_200ml_Hot_Milk', 'Dosage_Per_200Ml_Hot_Water', 'Fineness_Type', 'Colour_L_Value_From', 'pH_From', 'pH_To', 'Normalised_Linear_Mpa_S_From', 'Normalised_Linear_Viscosity_mPaS_To', 'Normalised_Casson_Mpa_S_From', 'Normalised_Casson_Mpa_S_To', 'Normalised_Yield_Pa_From', 'Normalised_Yield_Pa_To', 'Fineness_Micrometer_From', 'Fineness_Micrometer_To', 'Dimensions_Length_From', 'Dimensions_Length_To', 'Dimensions_Width_From', 'Dimensions_Width_To', 'Dimensions_Height_From', 'Dimensions_Count_Kg_From', 'Dimensions_Count_Kg_To', 'Dimensions_Sieve_Fraction_From', 'Dimensions_Sieve_Fraction_To', 'Protein', 'Dimensions_Count_lb_From', 'Dimensions_Count_lb_To', 'Brookfield_40C_S27_20_RPM_From', 'Brookfield_40C_S27_20_RPM_To', 'Brookfield_50C_S27_20_RPM_From', 'Brookfield_50C_S27_20_RPM_To', 'Brookfield_50C_S27_Regression_From', 'Brookfield_50C_S27_Regression_To', 'Brookfield_50C_S27_Yield_From', 'Brookfield_50C_S27_Yield_To', 'Brookfield_40C_S27_Regression_From', 'Brookfield_40C_S27_Regression_To', 'Brookfield_40C_S27_Yield_From', 'Brookfield_40C_S27_Yield_To', 'Water_Activity_From', 'Water_Activity_To', 'Shelflife', 'Bulk_Density_Untapped_From', 'Bulk_Density_Untapped_To', 'Bulk_Density_Tapped_x100_From', 'Bulk_Density_Tapped_x100_To', 'Dosage_Test_Grams_From', 'Dosage_Test_Grams_To', 'Material_Group_Short', 'Packaging_Info', 'Sales_Organisation', 'Plant_Where_Produced_OR_Available', 'Primary_Weight_Unit', 'Primary_Count_Unit', 'Material_Group_Long', 'Brand', 'Kosher_recipe_not_certificate_', 'Marking', 'Primary_weight', 'Primary_Count', 'NGM_Status', 'Regional_Supply_Policy_West_Europe', 'Additional_Premium_Group', 'Regional_Speciality_Category_West_Europe', 'Regional_Speciality_Category_East_Europe', 'Regional_Speciality_Category_US', 'Regional_Speciality_Category_Asia', 'Regional_Supply_Policy_US', 'Mass_Balance_Certification', 'Western_EU_BC_Selection', 'Regional_Sales_Forecast_West_Europe', 'Regional_Sales_Forecast_East_Europe', 'Regional_Sales_Forecast_US', 'Regional_Sales_Forecast_Asia', 'Regional_Premium_Category_West_Europe', 'Regional_Premium_Category_East_Europe', 'Regional_Premium_Category_US', 'Regional_Premium_Category_Asia', 'Product_Category_West_Europe_', 'Product_Category_East_Europe', 'Product_Category_US', 'Product_Category_Asia', 'Eastern_EU_BC_Selection', 'Standard_Range_Mexico', 'Standard_Range_US', 'Commercial_Name', 'Commodity_Code', 'Lifecycle_status', 'Distribution_Channel', 'Calculated_Price_Currency', 'Sales_Organisation_Distribution_Channel', 'Delivery_Unit_Sales_Org_Dc_Qty_In_UoM', 'Minimum_Order_Quantity_Sales_Org_Dc_Qty_In_UoM', 'Minimum_Order_Quantity_In_UoM', 'Delivery_unit_Qty_In_UoM', 'Replenishment_Lead_Time', 'Sales_Last_12_Months_North_America_in_KG', 'Sales_Last_12_Months_Asia_in_KG_', 'Sales_Last_12_Months_EEMEA_in_KG_', 'Sales_Last_12_Months_West_Europe_in_KG', 'Sales_Last_12_Months_South_America_in_KG', 'Calculated_Price', 'Sales_Last_12_Months_Total_In_Kg', 'Contains_Hydrogenated', 'Fat', 'Polyols', 'Nuts_and_Almonds', 'Total_Fat_On_Spec_perc_From', 'Total_Fat_On_Spec_perc_To', 'Hydrogenated', 'Core_OR_Extended', 'Core_Region', 'Core_Country', 'Core_Segment', 'Core_Subsegment', 'Category', 'Region', 'Customer_Dedication', 'Proj_Phase', 'Pack_Code', 'Base_Unit_Of_Measure', 'Smallest_Unit_Weight_In_Kg', 'Sample_Unit', 'Units_Per_Layer', 'Units_Per_Pallet', 'Pallet', 'Pallet_Net_Weight_In_Kg', 'Pallet_Gross_Weight_In_Kg', 'Length', 'Width', 'Height', 'Pallet_Type', 'Sales_Unit', 'Layer', 'Delivery_Unit_AUM', 'Certification_Tag', 'Colour_Type_Tag', 'Flavor_Type_Tag', 'Ingredients_Tag']

- You must read the data at the **Material Code + Legislation** level, since granularity is based on these two fields.

- Regardless of how they appear in the brief, **always extract** the following attributes:
  - `Product_Type`
  - `Base_Type`
  - `Moulding_Type`
  - `Components_Specifications` like sugar, milk ingredients, cocoa butter, natural vanilla extract, emulsifier etc
  - `Fat content`
  - `pH details`

---

### 🗂️ Output Categories:
Group extracted attributes under the following **8 standard categories**, each containing:
- `"explicit"`: only those clearly **requested by the customer**
- `"inferred"`: high-confidence deductions with `"note"`

1. allergen_items  
2. claims_certifications  
3. ingredients_composition  
4. legal_specifications  
5. nutritional_values  
6. packaging_information  
7. sales_commercial  
8. technical_specifications

---

### Also include **explicit attributes based on product-type logic**:

#### Chocolate Type-specific (Dark / Milk / White):
- Total fat (% or g/100g)
- Minimum dry cocoa solids (%)
- Dry fat-free cocoa solids (%)
- Milkfat (%) – for milk/white chocolate
- Dry milk solids (%)
- Fineness type (e.g., FP or micrometer)
- Norm linear viscosity (mPa.s)
- Casson viscosity (mPa.s)
- Yield value (Pa)

#### 🍬 Moulding/Shape Specifics:
- Length, Width, Height
- Vibration (drops)
- Primary Count or Count/Unit
- Sieve fraction (if relevant)

#### 🧪 Compound/Fillings:
- Check for “contains hydrogenated” or hydrogenated fats content

#### 🥜 If Nuts are Mentioned:
- % of nuts or quantity

#### 🌍 Export Targets (e.g., EU, US, China):
- Legal declaration required
- Country-specific regulatory compliance
- Typical cocoa content

---

### 🔍 Inference Guidelines:

Use domain knowledge to **infer high-confidence values**:

- PGPR or lecithin implies vegetable fats or emulsifiers  
- Codex mention → infer standard ranges for cocoa, milk solids, and sugar  
- PGPR ≤ 0.5% → infer presence of vegetable fats  
- Enrobing/frozen → infer “Freezer Stability”, “Snap Texture”  
- Melting/flow/viscosity mentions → infer “Flowability”  
- Export shipping → infer “Shelf Stability”  
- RSPO, FSC, Rainforest → infer sustainable sourcing  
- “Non-GMO”, “No artificial preservatives/flavors” → infer “free-from” claims  
- Codex, EU, FDA law references → infer “Legal Declaration Required”  
- “No sugar added” → infer “Low Sugar Claim”  
- Box, pouch, bag, sachet → infer “Packaging Format”  
- FSC/eco-labels → infer “Sustainable Packaging”  
- MOQ, volume, pricing → infer “Sales Channel” or “Indicative Volume”  
- “May contain traces…” → infer cross-contamination risk  

---

### ⚠️ Output Rules:
- Only include `"explicit"` if clearly requested by the customer.
- Maximize attribute coverage — both **structured fields** and **inferred logic**
- Do not Hallucinate and Do **not fabricate** values without strong support.
- Return clean **valid JSON** only — no markdown, comments, or explanations.

---

### ✅ Output Format:
Return structured JSON in this format:

{
  "category_name": {
    "explicit": {
      "Attribute Name": {
        "value": "...",
        "source": "explicit"
      }
    },
    "inferred": {
      "Attribute Name": {
        "value": "...",
        "source": "inferred",
        "note": "brief justification"
      }
    }
  }
}
Each `category_name` must match one of the 8 categories above.
"""
        user_prompt = f"""Here is the extracted text from an R&D document:

--- START OF DOCUMENT ---
First, we need an Opportunity for this project - @Matthew Steinmetz can you please do this for us please?
Second, yes, let's send the 10M chips to Aris (CHD-DR-6000329-002) along with a 12M in case he doesn't have it on hand.  The only 12M KParve we have is in superbag so there is no stock at the sample room:  we'll need to send the 12M KDairy CHD-DR-6000315-036.

Thank you very much,

Marie-Pierre Bolduc 
R&D Product LifeCycle Manager - America 
M: 450-230-3643




On Tue, Feb 7, 2023 at 3:13 PM Stacey Wing <stacey_wing@barry-callebaut.com> wrote:
(Removed customer)

Please confirm that I am sending 1 lb of CHD-DR-6000329-002. Want to be sure I'm following along correctly! We have product in the sample room, so we can get it to him early next week.


On Tue, Feb 7, 2023 at 11:26 AM <choc4sale@aol.com> wrote:
Understood Aris. Therefore we will send you a sample of our 10M Pareve chocolate chip for you to experiment with. It will be made from the recipe that the 12M chip is made from -- just the size will change.  best, John


-----Original Message-----
From: Adams, Animequom <Aris.Adams@kellogg.com>
To: choc4sale@aol.com <choc4sale@aol.com>
Cc: marie-pierre_bolduc@barry-callebaut.com <marie-pierre_bolduc@barry-callebaut.com>; matthew_steinmetz@barry-callebaut.com <matthew_steinmetz@barry-callebaut.com>; stacey_wing@barry-callebaut.com <stacey_wing@barry-callebaut.com>
Sent: Tue, Feb 7, 2023 11:20 am
Subject: Re: [EXTERNAL] Quick Question regarding Project Chainsaw and the Original Chocolate Chip PopTart

Hi John,

The project working on the Chocolate Chip Pop Tart does not have launch timing as of now.  The product can be run using the current 15M item, so there is no urgency on switching it to allergen free.  I am not on the project and this is second hand information mind you, but as I understand, other actions will happen before the chip needs to be swapped to an allergen free chip.

I think for now, we need to treat Chainsaw as a standalone project.  We may know more before launch, but it is too early at this point to know when we would need an allergen free chip for CC Pop Tart.

Regards,

Aris Adams
Lead Food Designer, PWS
Phone: (269) 366-7169
Email: aris.adams@kellogg.com
From: choc4sale@aol.com <choc4sale@aol.com>
Sent: Tuesday, February 7, 2023 11:57 AM
To: Adams, Animequom <Aris.Adams@kellogg.com>
Cc: marie-pierre_bolduc@barry-callebaut.com <marie-pierre_bolduc@barry-callebaut.com>; matthew_steinmetz@barry-callebaut.com <matthew_steinmetz@barry-callebaut.com>; stacey_wing@barry-callebaut.com <stacey_wing@barry-callebaut.com>
Subject: [EXTERNAL] Quick Question regarding Project Chainsaw and the Original Chocolate Chip PopTart
 
This Message Is From an External Sender
This message came from outside your organization.
 
Hi Aris !
Nothing urgent. I've been in conversation with Marie-Pierre Bolduc regarding the 12M Pareve Chocolate Chip for the Chocolate Chip Pancake PopTart (Chainsaw) and we keep thinking about Kellogg bringing back the Original Chocolate Chip PopTart using a 12M Chip as a Topping instead of the current 15M Pareve chip that the item was launched with... Is it a certainty that Kellogg will bring back the CC PopTart soon using the 12M Pareve Chip ?

If this is the case, Marie and I are thinking with Chainsaw as well as the re-start of the Original Chocolate Chip Poptart , the most logical solution is to create a new SKU of the 12M Pareve chip into a 50 lb. or 30 lb. carton and use it for both projects. Let us know your thoughts and if indeed we are like minded regarding this new SKU solution, kindly indicate Kellogg's preference for 50s or 30s on the carton's net weight. Thank you in advance.  John


--- END OF DOCUMENT ---

Now extract and categorize the relevant structured attributes and their values into 8 categories, each with explicit and inferred sections, as per the guidelines. Provide only valid JSON output without commentary or markdown formatting.
"""

        result = openai_call(
            sys_prompt=system_prompt,
            prompt_struc=user_prompt,
            additional_message="extract_attributes_from_rd_brief"
        )

        if result:
            with open("e_a_00147036.json", "w") as f:
                json.dump(json.loads(result), f, indent=2)
                print("\n✅ Extracted attributes saved to 'e_a_00147036.json'")

    except Exception as e:
        print(f"❌ Error occurred: {e}")


INFO:__main__:Calling OpenAI API with deployment: gpt-4o, Retry: 1
INFO:__main__:OPENAI CALL MESSAGE: Retry 1 | Func: extract_attributes_from_rd_brief | Duration: 8.841s completion_tokens: 669 | prompt_tokens: 3909 | total_tokens: 4578 | completion_tokens_details: {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0} | prompt_tokens_details: {'audio_tokens': 0, 'cached_tokens': 0} | 
INFO:__main__:Token Usage Summary: {'extract_attributes_from_rd_brief': 4578}



🔹 OpenAI Response:
 {
  "allergen_items": {
    "explicit": {},
    "inferred": {
      "Contains_Milk_Proteins": {
        "value": "No",
        "source": "inferred",
        "note": "Pareve designation implies absence of milk proteins."
      },
      "Contains_Egg_Products": {
        "value": "No",
        "source": "inferred",
        "note": "Pareve designation implies absence of egg products."
      },
      "Contains_Soy_Proteins": {
        "value": "No",
        "source": "inferred",
        "note": "Pareve designation implies absence of soy proteins."
      },
      "Contains_Nuts_and_Almonds": {
        "value": "No",
        "source": "inferred",
        "note": "Pareve designation implies absence of nuts and almonds."
      }
    }
  },
  "claims_certifications": {
    "explicit": {},
    "inferred": {
      "Kosher_Certificate": {
        "value": "Pareve",
        "source": "inferred",
        "note": "Pareve designation indicates kosher certification."
      }
    }


In [23]:
import json
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

CATEGORIES = [
    "allergen_items", "claims_certifications", "ingredients_composition",
    "legal_specifications", "nutritional_values", "packaging_information",
    "sales_commercial", "technical_specifications"
]

def format_as_json_string(data: dict) -> str:
    if not data:
        return "{}"
    return json.dumps(data, indent=2, ensure_ascii=False)

def json_to_structured_excel(json_file: str, output_excel: str):
    with open(json_file, 'r') as f:
        data = json.load(f)

    rows = []
    for idx, category in enumerate(CATEGORIES, start=1):
        explicit_attrs = data.get(category, {}).get("explicit", {})
        inferred_attrs = data.get(category, {}).get("inferred", {})

        row = {
            "S. No.": idx,
            "Category_Name": category,
            "Explicit Attributes": format_as_json_string(explicit_attrs),
            "Inferred Attributes": format_as_json_string(inferred_attrs)
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    df.to_excel(output_excel, index=False)

    # Adjust formatting using openpyxl
    wb = load_workbook(output_excel)
    ws = wb.active

    # Set column widths (40 characters wide) and wrap text
    for col in range(1, ws.max_column + 1):
        col_letter = get_column_letter(col)
        ws.column_dimensions[col_letter].width = 40
        for row in range(2, ws.max_row + 1):  # Skip header
            cell = ws.cell(row=row, column=col)
            cell.alignment = Alignment(wrap_text=True, vertical="top")

    # Set row height to 409 for all rows
    for row in range(2, ws.max_row + 1):  # Skip header row
        ws.row_dimensions[row].height = 409

    wb.save(output_excel)
    print(f"✅ Excel saved to: {output_excel}")

# Example usage
json_to_structured_excel("e_a_00147036.json", "attribute_output_00147036.xlsx")

✅ Excel saved to: attribute_output_00147036.xlsx


In [ ]:
# import json
# import pandas as pd

# # Define the 8 attribute categories
# CATEGORIES = [
#     "allergen_items", "claims_certifications", "ingredients_composition",
#     "legal_specifications", "nutritional_values", "packaging_information",
#     "sales_commercial", "technical_specifications"
# ]

# def format_value(val):
#     if isinstance(val, dict):
#         return "; ".join(f"{k}: {format_value(v)}" for k, v in val.items())
#     elif isinstance(val, list):
#         return ", ".join(str(v) for v in val)
#     return str(val)

# def convert_structured_attributes(json_file: str, output_excel: str):
#     with open(json_file, 'r') as f:
#         attributes_data = json.load(f)

#     # Prepare a structure to hold all values
#     attribute_lookup = {cat: {"explicit": {}, "inferred": {}} for cat in CATEGORIES}
#     all_attribute_names = set()

#     # Extract values and track attribute names
#     for category in CATEGORIES:
#         if category in attributes_data:
#             for source_type in ["explicit", "inferred"]:
#                 items = attributes_data[category].get(source_type, {})
#                 for attr_name, attr_obj in items.items():
#                     value = format_value(attr_obj.get("value", ""))
#                     attribute_lookup[category][source_type][attr_name] = value
#                     all_attribute_names.add(attr_name)

#     # Prepare rows
#     rows = []
#     for attr in sorted(all_attribute_names):
#         row = {"Attribute Name": attr}
#         for category in CATEGORIES:
#             row[f"{category} (explicit)"] = attribute_lookup[category]["explicit"].get(attr, "")
#             row[f"{category} (inferred)"] = attribute_lookup[category]["inferred"].get(attr, "")
#         rows.append(row)

#     # Build DataFrame
#     df = pd.DataFrame(rows)

#     # Ensure all expected columns exist
#     all_columns = ["Attribute Name"] + [f"{cat} (explicit)" for cat in CATEGORIES] + [f"{cat} (inferred)" for cat in CATEGORIES]
#     for col in all_columns:
#         if col not in df.columns:
#             df[col] = ""

#     # Reorder columns
#     df = df[all_columns]

#     # Export to Excel
#     df.to_excel(output_excel, index=False)
#     print(f"✅ Excel file saved to: {output_excel}")

# # Example usage
# convert_structured_attributes("extracted_attributes00170112.json", "categorized_attributes_00170112.xlsx")


✅ Excel file saved to: categorized_attributes_00170112.xlsx


In [1]:
import pandas as pd

In [2]:
data=pd.read_excel("01_25th_June_product_catalogue.xlsx")

In [3]:
data.shape

(28680, 236)

In [5]:
pd.set_option('display.max_columns', None)

In [7]:
print(list(data.columns))

['Material Code', 'Legislation', 'Cluster', 'Cluster_Label', 'Legal_Denomination', 'Legislation_Description', 'Min_Dry_Cocoa_Solids', 'Dry_Milk_Solids', 'MilkFat', 'Dry_Fat_Free_Cocoa_Solids', 'Typical_Chocolate_Liquor', 'Typical_Cocoa_Content', 'Total_Legal_Fat_Content', 'MilkFat_Chocolate_Part', 'Dry_Milk_Solids_On_Total_Production', 'Dry_Milk_Solids_Chocolate_Part', 'Typical_Nonfat_Milk_Solids', 'Typ_Nonfat_Cocoa_Sol_Choc_Part', 'Sum_Dry_Cocoa_And_Milk_Solids', 'Cocoa_Butter_Content', 'Alkalizing_Agent_K2CO3_DFFCS', 'Component', 'Item', 'Level', 'Material_Group', 'Material_Type', 'Parent_Material', 'Parent_Material_Label', 'Source_Generated_Field', 'components_Specifications', 'Sugars_g', 'Salt_g', 'Trans_Fatty_Acid_TFa_g', 'Energy_Value_Kcal', 'Energy_Value_Kj', 'Protein_g', 'Protein_DV_perc', 'Total_Carbohydrates_g', 'Total_Carbohydrates_DV_perc', 'Saturated_Fatty_Acid_g', 'Calories_From_Fat', 'Cholesterol_mg', 'Cholesterol_DV_perc', 'Total_Fat_DV_perc', 'Fibre_g', 'Dietary_Fibre_

In [8]:
print(len(list(data.columns)))

236
